In [1]:
import epidag as dag
import logging
import sys

logging.basicConfig(level=logging.INFO, stream=sys.stdout)

In [2]:
scr = '''
PCore test {
    al = 1
    be = 1
    prob ~ beta(al, be)
    x ~ binom(n, prob)    
}
'''

bn = dag.bn_from_script(scr)

In [3]:
data = [
    {'id': 1, 'n': 10, 'x': 4},
    {'id': 2, 'n': 20, 'x': 7}
]

dm = dag.as_data_model(bn, data)

In [4]:
p = dm.sample_prior()
dm.evaluate_likelihood(p)

-3.9601029282494236

In [5]:
fit = dag.fitting.SIR(dm)
fit.fit(1000)
fit.report_posterior()

2018-04-04 12:19:59,924 | INFO : Initialising
2018-04-04 12:19:59,931 | INFO : Sampling-Importance
2018-04-04 12:20:03,207 | INFO : Resampling


,al,be,prob
count,1000.0,1000.0,1000.000000
mean,1.0,1.0,0.376520
std,0.0,0.0,0.082066
min,1.0,1.0,0.160701
25%,1.0,1.0,0.322006
50%,1.0,1.0,0.372832
75%,1.0,1.0,0.424377
max,1.0,1.0,0.662095


In [6]:
fit = dag.fitting.MCMC(dm)
fit.fit(1000)
fit.report_posterior()

2018-04-04 12:20:08,921 | INFO : Initialising
2018-04-04 12:20:08,930 | INFO : Burning in
2018-04-04 12:20:11,283 | INFO : Gathering posteriori
2018-04-04 12:20:15,871 | INFO : Completed


,al,be,prob
count,1000.0,1000.0,1000.000000
mean,1.0,1.0,0.375551
std,0.0,0.0,0.081373
min,1.0,1.0,0.143048
25%,1.0,1.0,0.322809
50%,1.0,1.0,0.368929
75%,1.0,1.0,0.430943
max,1.0,1.0,0.624569


In [7]:
fit = dag.fitting.ABC(dm)
fit.fit(100)
fit.report_posterior()

2018-04-04 12:20:18,467 | INFO : Initialising
2018-04-04 12:20:18,672 | INFO : Testing threshold
2018-04-04 12:20:19,024 | INFO : Fitting
2018-04-04 12:20:24,875 | INFO : Gathering posteriori


,al,be,prob
count,100.0,100.0,100.000000
mean,1.0,1.0,0.363761
std,0.0,0.0,0.015218
min,1.0,1.0,0.340626
25%,1.0,1.0,0.350148
50%,1.0,1.0,0.361667
75%,1.0,1.0,0.377464
max,1.0,1.0,0.394594


In [5]:
scr = '''
PCore ex {
    sun ~ unif(0, 1)
    b0 = 1
    heat ~ exp(sun)
    x1 ~ norm(0, 1)
    x2 ~ norm(0, 1)
    b1 ~ unif(10, 20)
    mu1 = b0 + b1*x1 + b2*x2
    mu2 = b0 + b1*x1
    y1 ~ norm(mu1, sd)
    y2 ~ norm(mu2, 1)
}

'''

bn = dag.bn_from_script(scr)
hie = {
    'g': ['sun', 'agent'],
    'agent': ['x1', 'x2','mu1', 'mu2']
}
sm = dag.as_simulation_core(bn, hie=hie, random=['x2'])


h = dag.form_hierarchy(bn, hie)

h.print()

g(sd, heat, b1, b2, sun, b0)
-- agent(mu2, x1, x2, mu1, y2, y1)


In [21]:
pc = sm.generate(exo={'sd'})
ag1 = pc.breed('a1', 'agent')

In [30]:
ag1.get_sampler('y1')()

KeyError: 'b2'

In [29]:
pc.get_sampler('heat')()

0.7138004370161006

In [19]:
ng = dag.form_hierarchy(bn, hie)
dag.analyse_node_type(bn, ng, True)


Group g
Exogenous    : ['sd', 'b2']
Can be fixed: ['b1', 'b0', 'sun', 'heat']
Can be random: ['sun', 'heat']
Can be actors: ['heat']
--Group agent
  Exogenous    : []
  Can be fixed: ['mu2', 'x1', 'x2', 'y2']
  Can be random: ['mu2', 'x1', 'x2', 'y2', 'y1', 'mu1']
  Can be actors: ['y2', 'y1']


{'agent': ([], [], ['mu2', 'x1', 'x2'], ['y2'], ['y1'], ['mu1']),
 'g': (['sd', 'b2'], ['b1', 'b0'], ['sun'], ['heat'], [], [])}